In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter1.util.importer import *
MODEL_PATH = 'model_saves/testing_model'
nist_path = "../../src/data_preprocess/dataset/UKDATA_cleaned.csv"

In [ ]:
assert time_horizon > 0, "Time horizon must be a positive integer"
    
df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = MonteCarloPipeline.Builder() \
        .add_data(df) \
        .set_cleaner(cleaner) \
        .set_normalizer_class(MinMaxNormalizer) \
        .set_splitter(splitter) \
        .set_sequencer_class(TimeSequencer) \
        .set_target_column(TARGET_COLUMN) \
        .set_model(model) \
        .set_optimizer(optimizer) \
        .set_batch_size(batch_size) \
        .set_seq_len(seq_len) \
        .set_worker_num(NUM_WORKERS) \
        .set_error(NRMSE) \
        .set_trainer(trainer) \
        .set_tuner_class(StdTunerWrapper) \
        .set_inference_samples(inference_samples) \
        .add_test_error(NR) \
        .build()

model.fit()
torch.save(model.state_dict(), MODEL_PATH + ".pth")

In [ ]:
from experiments.iter1.util.evaluate import evaluate_model

import numpy as np
import pandas as pd
from src.util.power_splitter import PowerSplitter
from src.util.plotly import plot_results
from src.pipelines.normalizers.min_max_normalizer import MinMaxNormalizer
from src.util.evaluator import Evaluator

df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = MonteCarloPipeline.Builder() \
        .add_data(df) \
        .set_cleaner(cleaner) \
        .set_normalizer_class(MinMaxNormalizer) \
        .set_splitter(splitter) \
        .set_sequencer_class(TimeSequencer) \
        .set_target_column(TARGET_COLUMN) \
        .set_model(model) \
        .set_optimizer(optimizer) \
        .set_batch_size(batch_size) \
        .set_seq_len(seq_len) \
        .set_worker_num(NUM_WORKERS) \
        .set_error(NRMSE) \
        .set_trainer(trainer) \
        .set_tuner_class(StdTunerWrapper) \
        .set_inference_samples(inference_samples) \
        .set_inference_dropout(inference_dropout) \
        .add_test_error(NRMSE) \
        .build()

        #.add_test_error(NMCRPS) \
        #.add_test_error(NMLSCV) \
model.load_state_dict(torch.load(MODEL_PATH + ".pth", weights_only=True))
model.test()


evaluate_model(model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, seq_len, time_horizon, TARGET_COLUMN, error, temp_boundary, 0.95)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/9 [00:00<?, ?it/s]

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.